In [ ]:
# cleanig code using spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import trim, col

def main():
    spark = SparkSession.builder.appName("CleanDataset").getOrCreate()

    # Input and output paths - update file name if needed
    parquet_output_path = "gs://dataset_nutrifuson/cleaned-data/cleaned_dataset"
    csv_output_path = "gs://dataset_nutrifusion/cleaned-data/csv_output"
    input_path = "gs://dataset_nutrifusion/input-data/70000_recipes.csv"
"

    df = spark.read.option("header", "true").option("inferSchema", "true").csv(input_path)


    # Trim whitespace from string columns
    string_cols = [f.name for f in df.schema.fields if f.dataType.simpleString() == "string"]
    for c in string_cols:
        df = df.withColumn(c, trim(col(c)))

    # Drop rows with any nulls
    df_clean = df.na.drop()

    # Drop duplicates
    df_clean = df_clean.dropDuplicates()

    # Save cleaned dataset as Parquet
    df_clean.write.mode("overwrite").parquet(parquet_output_path)

    # Also save cleaned dataset as CSV (with header)
    df_clean.write.mode("overwrite").option("header", "true").csv(csv_output_path)

    print(f"Cleaned data saved as Parquet to: {parquet_output_path}")
    print(f"Cleaned data saved as CSV to: {csv_output_path}")

    spark.stop()

if __name__ == "__main__":
    main()


1st created a project in gcs as Nutrifusion_project
then created a bucket named dataset_nutrifusion


In [ ]:
# authenticate and onnect set the project using this
gcloud auth login
gcloud config set project nutrifusion-drive-2025


In [ ]:
#set the region
gcloud config set dataproc/region us-central1


In [ ]:
#checked the cluster status
gcloud dataproc clusters list
#this was my specific cluster
gcloud dataproc clusters describe recipe-cluster --region=us-central1



In [ ]:
# uploaded dataset the cleanig code to gcs
gsutil cp "D:\Downloads\sparkjob_2.py" gs://dataset_nutrifusion/scripts/


In [ ]:
#uploaded the dataset to the gcs using sdk
gsutil cp D:\OneDrive\Documents\DSMM\TERM 3\capstone [project\70000_recipes.csv gs://dataset_nutrifusion/input-data/

In [ ]:
# give me list of files in cluster
gsutil ls gs://dataset_nutrifusion/cleaned-data/


In [ ]:
#sparkjob in dataproc
gcloud dataproc jobs submit pyspark D:\Downloads\sparkjob_2.py --cluster=recipe-cluster --region=us-central1
Job [4c2fa67d5c664c50a9ff022de222e5a1] submitted.

In [ ]:
#code to convert the parquet to csv
from pyspark.sql import SparkSession

# Step 1: Start Spark session
spark = SparkSession.builder.appName("ParquetToCSV").getOrCreate()

# Step 2: Read Parquet data from GCS
df = spark.read.parquet("gs://dataset_nutrifusion/cleaned-data/cleaned_dataset/")

# Step 3: Convert to CSV and save back to GCS (single file)
df.coalesce(1).write.csv("gs://dataset_nutrifusion/cleaned-data/csv_output", header=True, mode="overwrite")

In [ ]:
#parquet in dataproc
gcloud dataproc jobs submit pyspark gs://dataset_nutrifusion/scripts/parquet.py --cluster=recipe-cluster --region=us-central1

In [ ]:
# list of datasets after runing the above
gsutil ls gs://dataset_nutrifusion/cleaned-data/csv_output/

In [ ]:
#code to download the csv file
gsutil cp gs://dataset_nutrifusion/cleaned-data/csv_output/part-00000-d356466d-8296-43fd-8965-709379ee4473-c000.csv